In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 3.8 MB/s 
     |████████████████████████████████| 596 kB 38.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.2 MB/s 
     |████████████████████████████████| 895 kB 47.3 MB/s 
     |████████████████████████████████| 77 kB 6.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip3 install lshash3

     |████████████████████████████████| 223 kB 4.7 MB/s 
  Created wheel for lshash3: filename=lshash3-0.0.8-py2.py3-none-any.whl size=10164 sha256=2e07f7217f2554c7a6573ec695758bd8e5a6bb7937252ea6b6c79cfd98d76576
  Stored in directory: /root/.cache/pip/wheels/65/72/01/dd1b56064187746f99aa560bbc491048575621ce6bb37cd05a
Successfully built lshash3


In [ ]:
from google.colab import drive
import pandas
from lshash import lshash
import sys, json
import pickle
drive.mount("/content/gdrive")
sys.path.append('/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt')

Mounted at /content/gdrive


In [ ]:
from Selectra import SELECTRA, pass_through_selectra
from transformers import ElectraTokenizer, ElectraModel, ElectraConfig
import torch
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

In [ ]:
model = SELECTRA(n_classes = 2)
model.load_state_dict(torch.load("/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt/LSH/best_model_state.bin",map_location=torch.device('cpu')))
model = model.to(device)
model.eval()

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SELECTRA(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=256, out_featur

In [ ]:
MODEL_NAME = "google/electra-small-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [ ]:
with open('/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt/LSH/sentences_dict.json', 'r') as fp:
    sent_dict = json.load(fp)

In [ ]:
with open("/content/gdrive/MyDrive/Colab Notebooks/SNLP projekt/LSH/lsh_model.pickle", 'rb') as pickle_file:
    lsh_model = pickle.load(pickle_file)

In [18]:
question = "What are some of the most controversial topics?"
embedding = pass_through_selectra(question, tokenizer, model, device).flatten()
response = lsh_model.query(embedding.to("cpu").detach().numpy(), num_results=3,distance_func = "euclidean" )

In [19]:
for i in range(len(response)):
  print(sent_dict[response[i][0][1]])

What are some of the most interesting psychological facts?
What are the most iconic images?
What are the most dangerous drugs?
